# Тестируем облачную функцию

### 1. Общие параметры

In [ ]:
# URL маршрута
map_url = 'https://www.yandex.ru/maps/?um=constructor%3A971ee70a42c0ded3a2bd90b4260df279fa36cdcd6d87473250011f3137a28970'

params = {'url': map_url,
          'lines': 'routes', #'routes', 'tracks' or 'track_segments'
          'placemarks': '1',
          'add_elevation': '0'}

### 2. Тестируем онлайн в облаке

In [ ]:
import requests

In [ ]:
# URL функции в файле в результате работы deploy.sh
with open('func-url.cfg', 'r') as file:
    url = file.read()
    
# GET
# В случае успеха выводится содержимое gpx: <?xml version="1.0" encoding="UTF-8"?>...

try:
    # Отправляем GET-запрос
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        # Выводим результат
        print(response.text)
    else:
        print(f"Функция вернула ошибку: статус-код {response.status_code}")
        
except Exception as e:
    print(f"Произошла ошибка: {e}")

In [ ]:
print(response.headers)

### 3. Тестируем локально

In [ ]:
from index import *

local_cache_dir = '../'

In [ ]:
params = {'queryStringParameters': params}

# В случае успеха выводится содержимое gpx: <?xml version="1.0" encoding="UTF-8"?>...
response = handler(params, None)
print(response['body'])

In [ ]:
print(response['headers'])